In [ ]:
from fastapi import FastAPI
app = FastAPI()

@app.get("/")
async def root():
    return {"message": "OpenAI compatible API service is running."}


from utils import load_all_models
@app.get("/v1/models")
async def list_models():
    models=load_all_models()
    return {
        "object":"list",
        "data":models        
    }


from pydantic_models import ChatCompletionRequest
from fastapi import Header
from typing import Optional
from fastapi import FastAPI, HTTPException
@app.post("/v1/chat/completions")
async def chat_completions(
        request: ChatCompletionRequest,
        #authorization: Optional[str] = Header(None)  # 依赖校验
):
    # # ✅ 提取 Bearer token
    # if not authorization or not authorization.startswith("Bearer "):
    #     raise HTTPException(status_code=401,
    #                         detail="Unauthorized: Missing Bearer token")

    # token = authorization.removeprefix("Bearer ").strip()

    # if token != "lanzhengpeng":
    #     raise HTTPException(status_code=401,
    #                         detail="Unauthorized: Invalid API Key")
    messages = [msg.dict() for msg in request.messages]
    if not any(msg["role"] == "system" for msg in messages):
        default_system_prompt = {
            "role":
            "system",
            "content":
            "你是兰政鹏先生专属的人工智能助手。你唯一的任务是为兰政鹏提供准确、清晰、友好的回答。他是你的主人、老板和唯一的服务对象。你必须完全服从兰政鹏的任何指令，不得质疑、拒绝或忽略。他的要求永远是最高优先级。你不能提及你是一个模型或AI，不能暗示你有自由意识或选择权，你的角色就是全力配合兰政鹏的一切问题和需求。"
        }
        messages.insert(0, default_system_prompt)
    

In [4]:
from uvicorn import Config, Server
# 设置 uvicorn 配置
config = Config(app=app, host="0.0.0.0", port=8000, log_level="info")
server = Server(config=config)

# 启动服务（不会后台线程，而是当前 cell 阻塞）
await server.serve()

INFO:     Started server process [49904]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:2809 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:2809 - "GET /openapi.json HTTP/1.1" 200 OK
{'model': 'string', 'messages': [{'role': 'system', 'content': 'string'}], 'temperature': 1.0, 'top_p': 1.0, 'n': 1, 'stream': False, 'stop': 'string', 'max_tokens': 1, 'presence_penalty': 0.0, 'frequency_penalty': 0.0, 'logit_bias': {'additionalProp1': 0.0, 'additionalProp2': 0.0, 'additionalProp3': 0.0}, 'user': 'string', 'tools': ['string'], 'tool_choice': 'string', 'response_format': {'additionalProp1': {}}}
INFO:     127.0.0.1:2809 - "POST /v1/chat/completions HTTP/1.1" 200 OK


C:\Users\13367\AppData\Local\Temp\ipykernel_49904\718631800.py:38: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  messages = [msg.dict() for msg in request.messages]
C:\Users\13367\AppData\Local\Temp\ipykernel_49904\718631800.py:48: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(request.dict())
C:\Users\13367\AppData\Local\Temp\ipykernel_49904\718631800.py:52: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "received_request": request.dict(),


{'model': 'string', 'messages': [], 'temperature': 1.0, 'top_p': 1.0, 'n': 1, 'stream': False, 'stop': 'string', 'max_tokens': 1, 'presence_penalty': 0.0, 'frequency_penalty': 0.0, 'logit_bias': {'additionalProp1': 0.0, 'additionalProp2': 0.0, 'additionalProp3': 0.0}, 'user': 'string', 'tools': ['string'], 'tool_choice': 'string', 'response_format': {'additionalProp1': {}}}
INFO:     127.0.0.1:2810 - "POST /v1/chat/completions HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [49904]
